# ch8 2 텍스트 분석 프로세스

텍스트 토큰화

In [5]:
text_sample='The Matrix is everywhere its all around us, here even in this room. \
You can see it out your window or on your television. \
You feel it when you go to work, or go to church or pay your taxes.'

In [8]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize, sent_tokenize
#여러 개 문장으로 된 입력 데이터를 문장별 단어 토큰화 하도록 함수 생성
def tokenize_text(text):
    #문장별로 자르자
    sentences = sent_tokenize(text)
    #자른 문장별 단어 토큰화 하자
    word_tokens = [word_tokenize(sentence) for sentence in sentences]
    return word_tokens
#생성된 함수를 사용해보자
word_tokens = tokenize_text(text_sample)
print(type(word_tokens),len(word_tokens))
print(word_tokens)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/hyeminchon/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


<class 'list'> 3
[['The', 'Matrix', 'is', 'everywhere', 'its', 'all', 'around', 'us', ',', 'here', 'even', 'in', 'this', 'room', '.'], ['You', 'can', 'see', 'it', 'out', 'your', 'window', 'or', 'on', 'your', 'television', '.'], ['You', 'feel', 'it', 'when', 'you', 'go', 'to', 'work', ',', 'or', 'go', 'to', 'church', 'or', 'pay', 'your', 'taxes', '.']]


스톱워드 제거

In [9]:
nltk.download('stopwords')
#stopwords 20개만 확인해보기
print('영어 stop words 개수:', len(nltk.corpus.stopwords.words('english')))
print(nltk.corpus.stopwords.words('english')[:20])

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hyeminchon/nltk_data...


영어 stop words 개수: 179
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his']


[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
stopwords=nltk.corpus.stopwords.words('english')
all_tokens=[]
#구했던 3개 문장별 토큰화된 단어 리스트에서 스톱워드 제거하는 반복문 만들기
for sentence in word_tokens:
    filtered_words=[]
    for word in sentence:
        #소문자로 모두 변환
        word=word.lower()
        #스톱워드에 해당 안되면 추가
        if word not in stopwords:
            filtered_words.append(word)
    all_tokens.append(filtered_words)
print(all_tokens)

[['matrix', 'everywhere', 'around', 'us', ',', 'even', 'room', '.'], ['see', 'window', 'television', '.'], ['feel', 'go', 'work', ',', 'go', 'church', 'pay', 'taxes', '.']]


Stemming & Lemmatization

In [12]:
from nltk.stem import LancasterStemmer
stemmer=LancasterStemmer()

print(stemmer.stem('working'), stemmer.stem('works'), stemmer.stem('worked'))
print(stemmer.stem('amusing'), stemmer.stem('amuses'), stemmer.stem('amused'))
print(stemmer.stem('happier'), stemmer.stem('happiest'))
#amuse의 원형 amus로 잘못 인식, happy 역시 비교형/최상급형 변형 캐치 실패

work work work
amus amus amus
happy happiest


In [19]:
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')

lemma = WordNetLemmatizer()
print(lemma.lemmatize('amusing','v'), lemma.lemmatize('amuses','v'), lemma.lemmatize('amused','v'))
print(lemma.lemmatize('happier','a'), lemma.lemmatize('happiest','a'))
#파라미터에 품사를 적어주므로 더 정확함

amuse amuse amuse
happy happy


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/hyeminchon/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# ch8 3 Bag of Words (BOW)

희소 행렬 변환: COO 형식

In [20]:
import numpy as np
dense=np.array([[3,0,1],[0,2,0]])

In [22]:
from scipy import sparse
#0이 아닌 데이터 추출
data=np.array([3,1,2])
#행과 열 위치를 각각 배열로 생성
row_pos=np.array([0,0,1])
col_pos=np.array([0,2,1])
#API 사용해서 COO형식으로 희소 행렬 생성
sparse_coo=sparse.coo_matrix((data, (row_pos, col_pos)))

#다시 밀집해보기
sparse_coo.toarray()

array([[3, 0, 1],
       [0, 2, 0]])

희소 행렬 변환: CSR 형식

In [23]:
dense2=np.array([[0,0,1,0,0,5],
                [1,4,0,3,2,5],
                [0,6,0,3,0,0],
                [2,0,0,0,0,0],
                [0,0,0,7,0,8],
                [1,0,0,0,0,0]])

In [24]:
from scipy import sparse
#0이 아닌 데이터 추출
data2=np.array([1,5,1,4,3,2,5,6,3,2,7,8,1])
#행과 열 위치를 각각 배열로 생성
row_pos=np.array([0,0,1,1,1,1,1,2,2,3,4,4,5])
col_pos=np.array([2,5,0,1,3,4,5,1,3,0,3,5,0])
#API 사용해서 COO형식으로 희소 행렬 생성
sparse_coo=sparse.coo_matrix((data2, (row_pos, col_pos)))
#행 위치 배열의 고유값 시작 위치 인덱스를 배열로
row_pos_ind=np.array([0,2,7,9,10,12,13])
#csr 형식으로 변환
sparse_csr=sparse.csr_matrix((data2, col_pos, row_pos_ind))

#다시 밀집해보기
print(sparse_csr.toarray())

[[0 0 1 0 0 5]
 [1 4 0 3 2 5]
 [0 6 0 3 0 0]
 [2 0 0 0 0 0]
 [0 0 0 7 0 8]
 [1 0 0 0 0 0]]
